In [34]:
import openai
import requests
import re
import os
import shutil
import time
from tqdm import tqdm

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [35]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

Download, Write, load the merged file

In [50]:
proj_name = 'architecture-samples_Diff'
merged_url = f'https://raw.githubusercontent.com/benymaxparsa/Kotlin_projects_commit_diff/main/{proj_name}-merged.txt'
paths_url = f'https://raw.githubusercontent.com/benymaxparsa/Kotlin_projects_commit_diff/main/{proj_name}-paths.txt'
dir_name = merged_url.split('/')[4]
print(dir_name)

Kotlin_projects_commit_diff


In [37]:
r = requests.get(merged_url, allow_redirects=True)
open('merged.txt', 'wb').write(r.content)

with open('merged.txt','r') as file:
    content = file.read()

scripts = content.split('<code block>')
print(len(scripts))
# print(scripts[0].strip())

455


In [38]:
r = requests.get(paths_url, allow_redirects=True)
open('paths.txt', 'wb').write(r.content)

with open('paths.txt','r') as file:
    paths = file.readlines()

print(len(paths))

455


In [ ]:
new_paths = []
for i in range(len(paths)):
    new_paths.append(''.join(paths[i].strip().split('.')[:-1]) + '.java')
print(new_paths[0])
len(new_paths)

In [39]:
def remove_markdown(text):
    text = text.replace('```java','')
    return text.replace('```','')
#     code_regex = r"java(.*?)"
#     match = re.search(code_regex,text, re.DOTALL)
#     if match is not None:
#         return match.group(1)
#     else:
#         return text

In [40]:
begin_dico = dict()
end_dico = dict()
def get_commit(path):
    return path.split('/')[2].split('-')[-1]

commits = [get_commit(c) for c in paths]
unique = set(commits)
for u in unique:
    begin_dico[u] = commits.index(u)
    end_dico[u] = len(commits) - commits[::-1].index(u)

In [53]:
def translate(begin,end):
    result = []
    index = begin
    for code in tqdm(scripts[begin:end]): 
        text = code.strip()
        prompt = f"""
        translate the kotlin code to java
        ```{text}```
        """
        try:
            response = get_completion(prompt)
        except Exception as e:
            response = f"line: {index}\nFile: {paths[index]}\nError: {e}"
            print(response)
        result.append(remove_markdown(response))
        
        index+=1
    return result

In [58]:
def save_files(result,begin,end):
    files = list(zip(result,new_paths[begin:end]))
    print(len(files))
    for file in files:
        path = os.path.dirname(file[1])
        if not os.path.exists(path):
            os.makedirs(path)
        with open(file[1].strip(),'w') as f:
            f.write(file[0])
    shutil.make_archive(f'{proj_name}-{begin}-{end}', 'zip', dir_name)
    shutil.rmtree(dir_name)

In [ ]:
commits = ["5dec53b4","0aba5dc1"]

for commit in commits:
    begin = begin_dico[commit]
    end = end_dico[commit]
    result = translate(begin,end)
    save_files(result,begin,end)

  0%|          | 0/4 [00:00<?, ?it/s]